In [23]:
import pandas as pd
import numpy as np
import re


## Loading data

First, load all of the data frames I'm going to need and check out their shape

In [623]:
seeds = pd.read_csv("march-machine-learning-mania-2016-v2/TourneySeeds.csv")
seeds.tail()

,Season,Seed,Team
2077,2016,Z13,1423
2078,2016,Z14,1453
2079,2016,Z15,1167
2080,2016,Z16a,1221
2081,2016,Z16b,1380


In [624]:
geog = pd.read_csv("march-machine-learning-mania-2016-v2/TourneyGeog.csv")
geog.drop(["lat", "lng", "host"], axis=1, inplace=True)
geog.head(5)

,season,daynum,wteam,lteam,slot
0,1985,136,1207,1250,R1W1
1,1985,136,1260,1233,R1W4
2,1985,136,1374,1330,R1W5
3,1985,136,1396,1439,R1W8
4,1985,136,1385,1380,R1X1


In [625]:
games = pd.read_csv("march-machine-learning-mania-2016-v2/TourneyCompactResults.csv")
#games = pd.concat([games, slots], ignore_index=True, axis=1)
print(games.shape)
games.head()



(1983, 8)


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [626]:
slots = pd.read_csv("march-machine-learning-mania-2016-v2/TourneySlots.csv")
slots = slots[(slots["Season"] < 2016)]
slots = slots.reset_index(drop=True)
print(slots.shape)
slots.head(10)

(1983, 4)


,Season,Slot,Strongseed,Weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12
5,1985,R1W6,W06,W11
6,1985,R1W7,W07,W10
7,1985,R1W8,W08,W09
8,1985,R1X1,X01,X16
9,1985,R1X2,X02,X15


In [627]:
print(games.shape, slots.shape)
#games = pd.concat([games, slots], axis=1).head()
games.head()

(1983, 8) (1983, 4)


,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [628]:
# do merging of games with seeds and slots 


games = pd.merge(games, seeds, left_on=["Season", "Wteam"], right_on=["Season", "Team"], copy=False)
games.rename(columns={"Seed":"Wseed"}, inplace=True)
print(games.shape)
games = pd.merge(games, seeds, left_on=["Season", "Lteam"], right_on=["Season", "Team"], copy=False)
games.rename(columns={"Seed":"Lseed"}, inplace=True)
print(games.shape)
games = pd.merge(games, geog, left_on=["Season", "Wteam", "Lteam"], right_on=["season", "wteam", "lteam"], copy=False)
print(games.shape)

games.drop(["wteam", "lteam", "Team_x", "Team_y", "Numot", "daynum", "season", "Wloc"], axis=1, inplace=True)

print(games.shape)

(1983, 10)
(1983, 12)
(1983, 17)
(1983, 9)


In [629]:
games = pd.merge(games, slots, left_on=["Season", "slot"], right_on=["Season", "Slot"], copy=False)
games.drop(["slot", "Wscore", "Lscore"], axis=1, inplace=True)
games.head(10)

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,Strongseed,Weakseed
0,1985,136,1116,1234,X09,X08,R1X8,X08,X09
1,1985,136,1120,1345,Z11,Z06,R1Z6,Z06,Z11
2,1985,138,1120,1242,Z11,Z03,R2Z3,R1Z3,R1Z6
3,1985,136,1207,1250,W01,W16,R1W1,W01,W16
4,1985,138,1207,1396,W01,W08,R2W1,R1W1,R1W8
5,1985,143,1207,1260,W01,W04,R3W1,R2W1,R2W4
6,1985,145,1207,1210,W01,W02,R4W1,R3W1,R3W2
7,1985,152,1207,1385,W01,X01,R5WX,R4W1,R4X1
8,1985,136,1229,1425,Y09,Y08,R1Y8,Y08,Y09
9,1985,136,1242,1325,Z03,Z14,R1Z3,Z03,Z14


In [630]:
# pre-process games

# drop early games
games = games[games["Daynum"] >  135]
games.rename(columns={"Strongseed":"seed1", "Weakseed": "seed2"}, inplace=True)

day_to_round = {136:1, 137:1, 138:2,139:2, 143:3, 144:3, 145:4, 146:4, 152:5,154:6}
round_to_points = {1:1, 2:2, 3:4, 4:8, 5:16, 6:32}
games["round"] = games["Daynum"].map(day_to_round)
games["bracket_points"] = games["round"].map(round_to_points)

games.head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
0,1985,136,1116,1234,X09,X08,R1X8,X08,X09,1,1
1,1985,136,1120,1345,Z11,Z06,R1Z6,Z06,Z11,1,1
2,1985,138,1120,1242,Z11,Z03,R2Z3,R1Z3,R1Z6,2,2
3,1985,136,1207,1250,W01,W16,R1W1,W01,W16,1,1
4,1985,138,1207,1396,W01,W08,R2W1,R1W1,R1W8,2,2


In [631]:
# make sure we have the right nr of games each year

num_games_by_day = games.groupby(["Season", "Daynum"])["Daynum", "Season"].agg(['count'])
games.groupby("Season").agg("count")["Daynum"]

Season
1985    63
1986    63
1987    63
1988    63
1989    63
1990    63
1991    63
1992    63
1993    63
1994    63
1995    63
1996    63
1997    63
1998    63
1999    63
2000    63
2001    63
2002    63
2003    63
2004    63
2005    63
2006    63
2007    63
2008    63
2009    63
2010    63
2011    63
2012    63
2013    63
2014    63
2015    63
Name: Daynum, dtype: int64

In [632]:
# do predictions by slots
def make_group(size):
    players = []
    for p in range(size):
        players.append(("p_" +str(p), p/size))
    return players

print(make_group(10))
    

[('p_0', 0.0), ('p_1', 0.1), ('p_2', 0.2), ('p_3', 0.3), ('p_4', 0.4), ('p_5', 0.5), ('p_6', 0.6), ('p_7', 0.7), ('p_8', 0.8), ('p_9', 0.9)]


In [633]:
games_2003 = games[games["Season"] == 2003]
print(games_2003.shape)
games_2003.head(20)

(63, 11)


,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
1137,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1,1
1138,2003,138,1112,1211,Z01,Z09,R2Z1,R1Z1,R1Z8,2,2
1139,2003,143,1112,1323,Z01,Z05,R3Z1,R2Z1,R2Z4,3,4
1140,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1,1
1141,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1,1
1142,2003,136,1143,1301,W08,W09,R1W8,W08,W09,1,1
1143,2003,136,1163,1140,X05,X12,R1X5,X05,X12,1,1
1144,2003,138,1163,1390,X05,X04,R2X4,R1X4,R1X5,2,2
1145,2003,136,1181,1161,Z03,Z14,R1Z3,Z03,Z14,1,1
1146,2003,138,1181,1141,Z03,Z11,R2Z3,R1Z3,R1Z6,2,2


In [634]:
def get_empty_bracket(tourney):
    by_round = {}
    for i in range(1,7):
        by_round[i] =  tourney[tourney["round"] == i]
        #print(by_round[i].shape)
    
    return by_round
empty = get_empty_bracket(games_2003)
empty[1].head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round,bracket_points
1137,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1,1
1140,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1,1
1141,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1,1
1142,2003,136,1143,1301,W08,W09,R1W8,W08,W09,1,1
1143,2003,136,1163,1140,X05,X12,R1X5,X05,X12,1,1


In [682]:
def doSub(s):
    #print(s)
    return int(re.sub("[^0-9]", "", s))
    

def do_picks(bracket, player):
    for rnd in bracket.keys():
        
        thisRound = bracket[rnd]

        #print("Round", rnd)
        #print(thisRound)
        if rnd == 1:
            #no need to get games from previous round'
            
            thisRound["seed1_num"] = thisRound["seed1"].apply(doSub)
            thisRound["seed2_num"] = thisRound["seed2"].apply(doSub)
            
            thisRound["pick"] = thisRound.apply(pickGame, axis=1, args=(player[1],))
            bracket[1] = thisRound
        else:
            # first get round and previous round
           
            prev = bracket[rnd-1]
            #print("prev")
            #print(prev)
            
            #merge slot with previous round winner
            thisRound = pd.merge(thisRound, prev.loc[:, ["Slot", "pick"]], left_on=["seed1"], right_on=["Slot"])
            
            thisRound.drop(["seed1", "Slot_y"], axis=1, inplace=True)
            thisRound.rename(columns={"pick":"seed1", "Slot_x": "Slot"}, inplace=True)
            
            #do again for other game
            thisRound = pd.merge(thisRound, prev.loc[:, ["Slot", "pick"]], left_on=["seed2"], right_on=["Slot"])
            thisRound.drop(["seed2", "Slot_y"], axis=1, inplace=True)
            thisRound.rename(columns={"pick":"seed2", "Slot_x":"Slot"}, inplace=True)
            
            
            thisRound["seed1_num"] = thisRound["seed1"].apply(doSub)
            thisRound["seed2_num"] = thisRound["seed2"].apply(doSub)
            #print(thisRound)
            # do picks
            thisRound["pick"] = thisRound.apply(pickGame, axis=1, args=(player[1],))
        
        # add picks to dict
        bracket[rnd] = thisRound
    return bracket
            
def pickGame(g, risk_factor):
    
    # adjust for seed difference, dont pick 16 over 1 as frequently as 10 over 7
    seed_diff = abs(g["seed1_num"] - g["seed2_num"])
    if seed_diff > 2:
        risk_factor /= seed_diff/2
    
    pick_upset = np.random.binomial(1, risk_factor)
    
    first_favorite = g["seed1_num"] < g["seed2_num"]
    #print(risk_factor, pick_upset,  first_favorite, g["seed1_num"] , g["seed2_num"] )
    if pick_upset and first_favorite or not pick_upset and not first_favorite:
        return g["seed2"]
    
    return g["seed1"]
    
    
    

In [683]:
this_bracket = get_empty_bracket(games_2003)
this_bracket = do_picks(this_bracket,  ("b",.5))
pd.concat(this_bracket.values()).tail(10)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,Daynum,Lseed,Lteam,Season,Slot,Wseed,Wteam,bracket_points,pick,round,seed1,seed1_num,seed2,seed2_num
5,144,X06,1268,2003,R3X2,X07,1277,4,X02,3,X02,2,X03,3
6,144,W10,1120,2003,R3W2,W03,1393,4,W07,3,W07,7,W03,3
7,144,X05,1163,2003,R3X1,X01,1400,4,X01,3,X01,1,X05,5
0,145,Z01,1112,2003,R4Z1,Z02,1242,8,Z07,4,Z01,1,Z07,7
1,145,Y01,1246,2003,R4Y1,Y03,1266,8,Y04,4,Y04,4,Y03,3
2,146,W01,1328,2003,R4W1,W03,1393,8,W05,4,W05,5,W07,7
3,146,X07,1277,2003,R4X1,X01,1400,8,X01,4,X01,1,X02,2
0,152,Y03,1266,2003,R5YZ,Z02,1242,16,Y04,5,Y04,4,Z07,7
1,152,X01,1400,2003,R5WX,W03,1393,16,X01,5,W05,5,X01,1
0,154,Z02,1242,2003,R6CH,W03,1393,32,Y04,6,X01,1,Y04,4


In [497]:
def is_upset(g):
    #whos the pick
    if g["pick"] == g["seed1"] and g["seed1_num"] > g["seed2_num"] or g["pick"] == g["seed2"] and g["seed1_num"] < g["seed2_num"]:
        return True
    return False
    

In [503]:
tournament = pd.concat(this_bracket.values())

tournament["is_upset"] =  tournament.apply(is_upset, axis=1)


round_to_points = {1:1, 2:2, 3:4, 4:8, 5:16, 6:32}

tournament["bracket_points"] = tournament["round"].map(round_to_points)
tournament["correct_pick"] = tournament["pick"] == tournament["Wseed"]
tournament["points_awarded"] = tournament["bracket_points"] * tournament["correct_pick"]

print(tournament.shape)
tournament.tail(15)

(63, 17)


,Daynum,Lseed,Lteam,Season,Slot,Wseed,Wteam,pick,round,seed1,seed1_num,seed2,seed2_num,is_upset,bracket_points,correct_pick,points_awarded
0,143,Z05,1323,2003,R3Z1,Z01,1112,Z05,3,Z16,16,Z05,5,False,4,False,0
1,143,Z03,1181,2003,R3Z2,Z02,1242,Z06,3,Z02,2,Z06,6,True,4,False,0
2,143,Y02,1338,2003,R3Y2,Y03,1266,Y11,3,Y02,2,Y11,11,True,4,False,0
3,144,W12,1139,2003,R3W1,W01,1328,W01,3,W01,1,W12,12,False,4,True,4
4,143,Y05,1458,2003,R3Y1,Y01,1246,Y13,3,Y09,9,Y13,13,True,4,False,0
5,144,X06,1268,2003,R3X2,X07,1277,X06,3,X07,7,X06,6,False,4,False,0
6,144,W10,1120,2003,R3W2,W03,1393,W07,3,W07,7,W14,14,False,4,False,0
7,144,X05,1163,2003,R3X1,X01,1400,X09,3,X09,9,X13,13,False,4,False,0
0,145,Z01,1112,2003,R4Z1,Z02,1242,Z06,4,Z05,5,Z06,6,True,8,False,0
1,145,Y01,1246,2003,R4Y1,Y03,1266,Y13,4,Y13,13,Y11,11,True,8,False,0


In [573]:
tournament["bracket_points"].sum()

192

In [511]:
def do_preds_for_group(group, games):
    res = []
    for player in group:
        this_bracket = get_empty_bracket(games)
        this_bracket = do_picks(this_bracket,  player)
        tournament = pd.concat(this_bracket.values())

        tournament["is_upset"] =  tournament.apply(is_upset, axis=1)


       
        tournament["correct_pick"] = tournament["pick"] == tournament["Wseed"]
        tournament["points_awarded"] = tournament["bracket_points"] * tournament["correct_pick"]
        
        res.append({"player": player[0], "upsets": tournament["is_upset"].sum(), "points" : tournament["points_awarded"].sum()})
        
        # delete this guys stuff
        del tournament
        del this_bracket
    return pd.DataFrame(res)



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [517]:
gps_100 = []

In [565]:
gps_100.append(do_preds_for_group(make_group(100), games_2003))
gps_100[len(gps_100)-1].sort_values("points", ascending=False).head(10)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,player,points,upsets
70,p_70,133,26
54,p_54,130,20
91,p_91,112,25
84,p_84,108,26
37,p_37,104,13
66,p_66,89,20
44,p_44,86,11
99,p_99,83,24
17,p_17,82,7
20,p_20,81,6


In [451]:
games_2003.head()

,Season,Daynum,Wteam,Lteam,Wseed,Lseed,Slot,seed1,seed2,round
1137,2003,136,1112,1436,Z01,Z16,R1Z1,Z01,Z16,1
1138,2003,138,1112,1211,Z01,Z09,R2Z1,R1Z1,R1Z8,2
1139,2003,143,1112,1323,Z01,Z05,R3Z1,R2Z1,R2Z4,3
1140,2003,136,1113,1272,Z10,Z07,R1Z7,Z07,Z10,1
1141,2003,136,1141,1166,Z11,Z06,R1Z6,Z06,Z11,1


In [684]:
def optimal_preds_for_group(games, size, iterations):
    winners = []
    for i in range(iterations):
        brackets = do_preds_for_group(make_group(size), games)
        winners.append(brackets.sort_values("points", ascending=False).head(1))
    return pd.concat(winners)



In [695]:
groups_of_10_2003 = optimal_preds_for_group(games_2003, 10, 100)
groups_of_10_2003.mean()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [696]:
groups_of_10_2003.mean()

points    96.70
upsets    15.75
dtype: float64

In [ ]:
groups_of_100_2003 = optimal_preds_for_group(games_2003, 100, 100)
groups_of_100_2003.mean()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea